In [1]:
import pandas as pd
import numpy as np
import gzip
import json
import re
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()


%matplotlib inline

# Section 1. Cleaning up the books data

Using the json file from UCSD which contains about 2.6M book entries, clean up the data.

This consists of:
1. Removing non-English and children books

This is achieved by extracting the books where language_id is not null or eng. Somehow this entry is not correctly labelled all the time, and English books do not necessarily have a correct language_id (it can be void). If child is found in the shelves names (or tags) remove them as well.

2. Cleaning tags

Tags are redundant and very messy. Previous EDA showed that the first most used 500 tags are enough to cover about 90% of the number of books (confirmer), and that after the third tag generally speaking more errors happen.
2.1 Remove non-informative tags such as to-read etc...
2.2 Recover abbreviations, ex ya for young-adult


In [2]:
# Some useful methods related to json files

# Reading the input json file
def load_data(inputdata):
    books = []
    with gzip.open(inputdata) as json_file:
        for line in json_file:
            book = json.loads(line)
            books.append(book)
    return books

# Extracting subset of the data
def split_data(inputdata, cutoff):
    books = []
    count = 0
    with gzip.open(inputdata) as json_file:
        for line in json_file:
            if count <= cutoff:
                book = json.loads(line)
                count += 1
                books.append(book)
    return books

# get data needed from json
def get_books_data(inputdata):
    books = []
    with open(inputdata) as json_file:
        for lines in json_file:
            book = {}
            line = json.loads(lines)
            names = []
            counts = []                   
            names = [elm['name'] for elm in line['popular_shelves']] 
            counts = [elm['count'] for elm in line['popular_shelves']]
            book['isbn'] = line['isbn']
            book['book_id'] = line['book_id']
            book['popular_shelves'] = names
            book['count_shelves'] = counts
            book['title'] = line['title']
            book['num_pages'] = line['num_pages']
            book['publication_year'] = line['publication_year']
            book['average_rating'] = line['average_rating']
            book['ratings_count'] = line['ratings_count']
            books.append(book)
    return books


## 1.1 Getting English books


Converting the json file into a pandas dataframe, total number of books in the database is: 2,360,655

In [3]:
def clean_data_language(inputdata, outputfile):
    eng_lan = ['en', 'enm', 'en-US', 'en-GB', '']
    child_books = ['child', 'children', 'children-s', 'childrens', 'kids-books', 'childrens-s-books']

    output = open(outputfile, 'w')
    with gzip.open(inputdata) as json_file:
        for line in json_file:
            book = json.loads(line)
            names = [elm['name'] for elm in book['popular_shelves']]
            if (book['language_code'] in eng_lan) and (not set(child_books).isdisjoint(names[:6]) == False):
                output.write('{}\n'.format(json.dumps(book)))
    output.close()



In [4]:
# First get english and non-children books
books_json = clean_data_language('goodreads_books.json.gz', 'books_en_nochild.json')

After removing these entries, the number of books is now up to: 1,156,654

In [5]:
books_en = get_books_data('books_en_nochild.json')
booksdf = pd.DataFrame(books_en)

In [6]:
len(booksdf)

1156654

## 2.1 Removing unwanted tags

The first step in order to avoid an overwhelming amount of data is to select the main populated genres.

In [16]:
# get appropriate tags
def get_genre(inputdata, outputfile):
    wanted = ['classics', 'classic', 'fantasy', 'romance', 'mystery', 'science-fiction', 'sci-fi',
             'scifi', 'business', 'economics']

    output = open(outputfile, 'w')
    
    with open(inputdata) as json_file:
        for line in json_file:
                book = json.loads(line)
                names = [elm['name'] for elm in book['popular_shelves'] if \
            bool(re.match('[^\x00-\x7F\x80-\xFF\u0100-\u017F\u0180-\u024F\u1E00-\u1EFF]', elm['name'])) == False]
                if set(wanted).isdisjoint(names) == False:
                    output.write('{}\n'.format(json.dumps(book)))

    output.close()

In [17]:
get_genre('books_en_nochild.json', 'books_genre.json')

Convert the resulting json file into a pandas DataFrame

In [18]:
books = get_books_data('books_genre.json')
booksdf = pd.DataFrame(books)

Next, we will remove unwanted tags, identified thus far and belonging to uninformative categories, such as to-read (most populated shelve with 67% of the tags).

In [8]:
def clean_data_nasty_tags(dfin):

    popular_shelves = dfin['popular_shelves']
    count_shelves = dfin['count_shelves']
    
    popnew = []
    cnew = []

    for elm, elmc in zip(popular_shelves, count_shelves):    
        to_remove = []
        for i in range(len(elm)):
            if (len(elm[i]) == 1) or (bool(re.match('.*book*', elm[i])) == True)\
            or (bool(re.match('.*read*', elm[i])) == True) or (bool(re.match('.*favorite*', elm[i])) == True)\
            or (bool(re.match('.*need*', elm[i])) == True) or (bool(re.match('.*own*', elm[i])) == True)\
            or (bool(re.match('.*shelve*', elm[i])) == True) or (bool(re.match('.*like*', elm[i])) == True)\
            or (bool(re.match('.*shelf*', elm[i])) == True) or (bool(re.match('.*buy*', elm[i])) == True)\
            or (bool(re.match('tbr', elm[i])) == True) or (bool(re.match('.*finish*', elm[i])) == True)\
            or (bool(re.match('.*kindle*', elm[i])) == True) or (bool(re.match('.*list*', elm[i])) == True)\
            or (bool(re.match('.*year*', elm[i])) == True) or (bool(re.match('.*audio*', elm[i])) == True)\
            or (bool(re.match('.*library*', elm[i])) == True) or \
            (bool(re.match('[^\x00-\x7F\x80-\xFF\u0100-\u017F\u0180-\u024F\u1E00-\u1EFF]', elm[i])) == True):
                to_remove.append(i) 
        
        popnew.append([elm[e] for e in range(len(elm)) if e not in to_remove])
        cnew.append([elmc[e] for e in range(len(elmc)) if e not in to_remove])
 
    return popnew, cnew

In [10]:
#df2 = booksdf[booksdf['isbn'] != '']

In [19]:
len(booksdf)

502673

In [20]:
df_copy = booksdf.copy()

In [14]:
list_tags = df2['popular_shelves'].to_list()
test = [e for elm in list_tags for e in elm]
#for i in test:
#    if i == 'business':
#        print(i)

In [21]:
popnew, cnew = clean_data_nasty_tags(booksdf)
new_column_p = pd.Series(popnew, name='popular_shelves', index=range(len(popnew)))
new_column_c = pd.Series(cnew, name='count_shelves', index=range(len(cnew)))


In [23]:
df2.update(new_column_p)
df2.update(new_column_c)

The resulting df is now cleaned of those tags, however business related books were also removed (TBD why...)

In [24]:
# get data needed to facilitate processing
def get_business_books(inputdata):
    books = []
    with open(inputdata) as json_file:
        for lines in json_file:
            book = {}
            line = json.loads(lines)
            names = []
            counts = []
            for elm in line['popular_shelves']:
                if (bool(re.match('.*business*', elm['name'])) == True):
                    names.append(elm['name'])
                    counts.append(elm['count'])                   
            if len(names) > 1: 
                book['isbn'] = line['isbn']
                book['book_id'] = line['book_id']
                book['popular_shelves'] = names
                book['count_shelves'] = counts
                book['average_rating'] = line['average_rating']
                book['title'] = line['title']
                books.append(book)
    return books

In [25]:
bbooks = get_business_books('books_genre.json')

business
business-design-support-creating
business-finance-economics
business-reading
business
businessman
business
business-books
business-econ
mgmt-business
better-business
business-life-business-comm
business
business
business
business
business-economics
business
business
business-psychology
small-business-success-stories
business---econ
business
business-marketing
business-finance-ceo-professiona
business
business
business
business
business
business
business-books
business-leadership
business-money
business-analysis
leadership-business
business-management
non-fic-business
v-business
business-analyst
want-to-read-business
business-book
business-money
business
business-etrepreneur
business
business
business-leadership
business
business
business
business
business-books
business-econ-psych
andrew-s-business-books
business
economics-business
business
business-management
small-business
business
business-books
business
non-fiction-business
business
small-business
business
business-books
b

business
business
business-economics
business
economics-business-invention
business-management-career
business-books
prosperity-and-business
business
business
business-books
business-money
business-finance
nonfiction-business
business-investing
all-business-etc
business
business
it-business
business-leadership
business-self-help
business-management
sexy-businessman
business
business
art-business
business-marketing
pk-marketing-business
business
l4-business
business-finance
business-finance-economics
economics-business
business
business
read-business-books
business-personal-brand
business-management
business-management-career
business
businesswoman
business
business
black-business
business
business
business
business-books
business-work
t-business
business-development
business-development-books
business-to-read
career---business
business
business-investing
andrew-s-business-books
eco-business-finance
business
business
business-books
to-read-business
business-leadership
650-business
busin

business
business
business
to-read-business-finance-economics
business-innovation-leadership
business
business-must-read
to-read-business-success-leadership
business-and-finance
business
bios-of-corps-businessmen-deals
business
business
home-based-businesses
businesswoman
business
business-finance-economics
business
business-books
business-economics-money
tic-business
business
global-business
business-books-to-read
legal-business-thriller
business-thrillers
biography_history_business
business
business
business
businessman
business
business-general
reviewed-for-business
business
business
business
library-business
business
management-business
to-read-business
business-leadership
newspaper-business
the-business
business
business-tycoon
business
business
business-economics
economics-business
business-finance
economics-finance-business
business-history
business-and-economics
business
business-books
business-marketing
business
business-stuff
business
business-economics
business-books
economi

business
business
business
business
business
business
business
business-books
business
business-how-to
business
business-owner
business
business-entrepreneur-career
business
business
business-finance-economics
business-history
business-profiles
business
business
business-book
business-and-self
business-world
marketing-business
business
business-economics-money
business
business-finance
business-economics
business-books
economics-business
business-and-economics
business
business-books
unfinished-business
business
business-books
business-career
business-books-to-read
businesswoman
business-economics
business-books
business
business-management
business-leadership
to-read-business
business-finance
business-to-read
business-self-help
business-economics
business-marketing
business-and-investing
business
self-help-business
business-finance-investing
business-and-management
business-management
business-career
business-management-career
business
business-work-career
business
business
business
b

business
business-economics
serious-business
business
business
business
business
business-books
business
business-marketing
business-and-investing
business-finance
business-economics-money
business
business-books-partially-read
business-books
business
film-business
business
economics-business-finance
business-and-finance
business
business-economy
business-musts
business-career
business
business
business
collins-business-essentials
shelfari-business-sw-engineering
shelfari-business
business-y
650-business
business-entrepreneur
business-leadership
toread-business
business
business-books
business-school
to-read-business
business-to-read
business-management
business-general
nasser6-business-2
business-schools
business-motivation
general-business
business-owner
business
business-and-economics
business-and-economics
business
business-books
business-and-training
business
business
business
business-woman
business
business-entrepreneurship
to-read-business
business-to-read
business-books
busine

...

family-business
business
business
non-fiction-business
business-novel
business-leadership-success
nonfiction-business
business-books-to-read
business-book-club
business
business
to-read-business
business-books
99-best-business-books
non-fic-business
business
business-self-help-to-read
business-history
musicbusiness
business-economics
business
economics-business
business-finance
business-and-economics
business-and-finance
business-economics
to-read-business
business-economy
business
business
business-investing
business-finance
business-books
business-economics-finance
history-business-politics
business-su-vc
business-personal-finance
business-finance-economics
finance-economics-business
eco-fin-business
business
business-non-fiction
business-or-leadership
to-read-business
finished-business-leadership
business-reading
women-business
small-business
pd-and-business
business
business-accounting
business-theory
business-economics
business
business
business-economics
economics-finance-busines

business
business-economics
economics-business
business-books
business-and-economics
business
business
business
business
business
business
business
t-business-life-management
sanjaybakshi-businessmodelshelf
business-models
business-entrepreneur-to-read
business-strategy
business-management
business-development
nonfiction-business-and-economics
business
business-books
business-management-career
music-business-entrepreneurship
business
business
economics-business-finance
economics-business-and-finance
finance-business
eco-business
business
business-owner
business-women
business-exec
business
business
business-finance
business-career
business-books
business-and-money
business
business-books
business
business-and-professional
business-sales-motivational
hero-businesman-businessowner
business
business-and-management
business
nonfiction-business-related
economics-business-and-finance
business-marketing
business-and-investing
to-read-business
business
business
business-finance
business
busine

business
work-business
business
read-non-business-related
business
business
heroine-falls-for-business-partner
hero-is-a-businessman
business
business
business-management
business
business-marketing
business-leadership
business-books
marketing-business-social-media
my-business
non-fiction-business
business-of-life
business-related
business
business-and-finance
business-finance-economics
economics-business
business
hot-billionaire-business-ceo-guys
business
business
business
business-books
business-entrepreneur
top-business-books
business-books-to-read
ft-mckinsey-best-business-books
business-marketing
my-bib-business
business-finance
business
business
business
business
business-books
to-read-business
business-finance
business-to-read
business
business
business
business
business
business
business
business-rec
family-business
businessman-or-businesswoman
business
business
business
business-and-economics
business
businessman
business-and-leadership
business
building-my-business
business-m

business
business-and-self-help
general-business
business-management
business-text
business
finance-business
business
business
business
productivity-business
business-finance
business
creative-business
business
business
business
business
business-books-3
businessman-or-businesswoman
business
business
business-books
to-read-business
business
business-101
business
business
business
business
business
eco-fin-business
eco-fin-business
marketing-business
business
economics-business
business-economy
business
business-mgmt
business-and-finance
business-owners
business-finance-economics
business
business
history-economic-business
business
business
business-books
to-read-business
business-management
business-novels
business
business-books
business-economics
business-book
business
business-books
business-economics
top-business-books
business-books-to-read
ft-mckinsey-best-business-books
business-books-of-the-decade
business-bucket
to-read-business
hireshai-s-business-book-reads
business
business

grown-folks-business
business
business
business
business-and-professional
business-and-success
business-economics-finance
business
business-finance
business-books
business-biographies
business-and-finance
business-motivation
business
business
business
business-books
to-read-business
business-marketing
business-management
business-and-marketing
business-to-read
entrepreneurship-business
business-skills
business-money
business-and-finance
business-self-help
not-so-serious-business
business
business-management
businessman
business
business-books
business
business
business-books
business-to-read
to-read-business
business-career
career-leadership-business
topic-business
career-business-code
business-entreprenuership
audiobook-business
business-technology-management
to-read-business-books
non-fiction-business
business
business
business
business
business
business
business
too-much-junkie-business-johnny
funny-business
business
business
business
business
career-business
business-marketing
busi

In [26]:
dfb = pd.DataFrame(bbooks)
dfb.head()

,average_rating,book_id,count_shelves,isbn,popular_shelves,title
0,4.00,18264569,"[1, 1, 1, 1]",1118727010,"[business, business-design-support-creating, b...",Investor Behavior: The Psychology of Financial...
1,3.38,19222476,"[13, 2, 1, 1, 1, 1]",,"[business, business-books, business-econ, mgmt...",Send
2,3.26,6690904,"[5, 1]",1608190234,"[business, business-economics]","Keynes: The Rise, Fall, and Return of the 20th..."
3,3.94,288186,"[6, 1, 1, 1]",0385508026,"[business, business-psychology, small-business...",Shameless Exploitation in Pursuit of the Commo...
4,5.00,34632203,"[1, 1]",,"[business, business-marketing]",A Practitioner's Guide to Account-Based Market...


In [27]:
alldf = df2.append(dfb)

/Users/yasminechebaro/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [29]:
len(alldf)

809441

Merging the business data with the cleaned dataframe, resulting in 809,441 book entries.

Next, only the first three tags are kept, and entries which do not have three tags are removed.

In [31]:
# First take only first 10 tags
def get_first_tags(dfin, cutoff):
    popular_shelves = dfin['popular_shelves']
    count_shelves = dfin['count_shelves']    
    popnew = []
    cnew = []

    for elm, elmc in zip(popular_shelves, count_shelves):    
        listp = [elm[e] for e in range(len(elm))][0:int(cutoff)]
        listc = [elmc[e] for e in range(len(elmc))][0:int(cutoff)]
        if len(listp) >= 3:
            popnew.append(listp)
            cnew.append(listc)
    
    return popnew, cnew

In [32]:
p, c = get_first_tags(alldf, 3)

In [33]:
new_column_p = pd.Series(p, name='popular_shelves', index=range(len(p)))
new_column_c = pd.Series(c, name='count_shelves', index=range(len(c)))
alldf.update(new_column_p)
alldf.update(new_column_c)

In [36]:
len(alldf)

809441

In [37]:
alldf.head()

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title
0,4.00,5333265,"[4, 3, 2]",0312853122,256,"[history, classics, level-4to5]",1984,3,W.C. Fields: A Life on Film
1,3.23,1333909,"[25, 9, 9]",0743509986,,"[fantasy, magic, avalon]",2001,10,Good Harbor
2,3.40,287140,"[1, 1, 1]",0850308712,,"[sports-romance, romance, f-f]",,15,Runic Astrology: Starcraft and Timekeeping in ...
3,4.13,287141,"[12, 11, 10]",1599150603,162,"[historical-fiction, mystery, historical-mystery]",2006,46,The Aeneid for Boys and Girls
4,4.38,378460,"[291, 187, 80]",0425040887,,"[philosophy, non-fiction, religion]",1979,12,The Wanting of Levine


In [42]:
def replace_tags(dfin):
    popular_shelves = dfin['popular_shelves']
    count_shelves = dfin['count_shelves']    
    
    popnew = []
    cnew = []

    for elm, elmc in zip(popular_shelves, count_shelves):    
        for i in range(len(elm)):
            elm[i].replace('classics', 'classic')
            elm[i].replace('scifi', 'science-fiction')
            elm[i].replace('sci-fi', 'science-fiction')
            elm[i].replace('ya', 'young-adult')

        popnew.append([elm[e] for e in range(len(elm))])
        cnew.append([elmc[e] for e in range(len(elmc))])
    
    return popnew, cnew

In [43]:
p, c = replace_tags(alldf)

In [44]:
new_column_p = pd.Series(p, name='popular_shelves', index=range(len(p)))
new_column_c = pd.Series(c, name='count_shelves', index=range(len(c)))
alldf.update(new_column_p)
alldf.update(new_column_c)

Let us test if the popular and count shelves are still OK.

In [45]:
df2[df2['book_id'] == '287141']

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title
3,4.13,287141,"[12, 11, 10, 7, 6, 5, 3, 3, 2, 2, 2, 2, 2, 1, ...",1599150603,162,"[historical-fiction, mystery, historical-myste...",2006,46,The Aeneid for Boys and Girls


In [46]:
alldf[alldf['book_id'] == '287141']

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title
3,4.13,287141,"[12, 11, 10]",1599150603,162,"[historical-fiction, mystery, historical-mystery]",2006,46,The Aeneid for Boys and Girls


In [47]:
list_tags = alldf['popular_shelves'].to_list()
list_tags_flat = [e for elm in list_tags for e in elm]
#for i in test:
#    if i == 'business':
#        print(i)

In [49]:
len(set(list_tags_flat))

761940

In [50]:
data = pd.DataFrame(zip(alldf['book_id'], alldf['popular_shelves']), columns=['book_id', 'popular_shelves'])
data.head()

,book_id,popular_shelves
0,5333265,"[history, classics, level-4to5]"
1,1333909,"[fantasy, magic, avalon]"
2,287140,"[sports-romance, romance, f-f]"
3,287141,"[historical-fiction, mystery, historical-mystery]"
4,378460,"[philosophy, non-fiction, religion]"


In [ ]:
shelves = data['popular_shelves'].apply(pd.Series)

In [ ]:
newdata = shelves.merge(data, right_index = True, left_index = True)

newdata.head()